In [1]:
%reset -f
from parflow import Run
from parflow.tools.fs import get_absolute_path
from parflow.tools.io import write_pfb, read_pfb
import numpy as np
from matplotlib import pyplot as plt
from parflow.tools.hydrology import calculate_water_table_depth, \
calculate_overland_flow_grid, calculate_overland_fluxes
# %matplotlib widget

In [3]:
nx = 20
ny = 5
nz = 20
dx = 5
dy = 0.2
dz = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, \
               0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.3, 0.1])
ztemp = np.cumsum(dz) - dz/2.
print(ztemp)

[0.25 0.75 1.25 1.75 2.25 2.75 3.25 3.75 4.25 4.75 5.25 5.75 6.25 6.75
 7.25 7.75 8.25 8.75 9.15 9.35]


In [4]:
Vx = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.velx.43800.pfb'))
Vy = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.vely.43800.pfb'))
Vz = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.velz.43800.pfb'))
Saturation = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.satur.43800.pfb'))
Porosity = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.porosity.pfb'))
print(Vx.shape, Vy.shape, Vz.shape)

(20, 5, 21) (20, 6, 20) (21, 5, 20)


In [5]:
#setup indices

indx = np.zeros((nz,ny,nx))
for i in range(nx):  
    indx[:,:,i] = np.ones((nz,ny)) * i
# print(indx[10,:,:])

indy = np.zeros((nz,ny,nx))
for j in range(ny):  
    indy[:,j,:] = np.ones((nz,nx)) * j
# print(indy[10,:,:])

indz = np.zeros((nz,ny,nx))
for k in range(nz):  
    indz[k,:,:] = np.ones((ny,nx)) * k
# print(indz[10,:,:])

In [6]:
#setup initial location
npcell = 40

initx = np.zeros((npcell,nz,ny,nx))
inity = np.zeros((npcell,nz,ny,nx))
initz = np.zeros((npcell,nz,ny,nx))

# generate random numbers
# uniform distribution between 0 and 1 with mean of 0.5 and std of 1/(2*sqrt(3))

for i in range(npcell):
    randx = np.random.uniform(low=0.0, high=1.0, size=(nz,ny,nx))
    randy = np.random.uniform(low=0.0, high=1.0, size=(nz,ny,nx))
    initx[i,:,:,:] = (indx + randx)*dx
    inity[i,:,:,:] = (indy + randy)*dy
    initz[i,:,:,:] = ztemp[:, np.newaxis, np.newaxis]
# initz[1,19,:,:]

In [7]:
#cal locid
Px = np.copy(initx)
Py = np.copy(inity)
Pz = np.copy(initz)

Plocx = np.floor(Px/dx)
Plocy = np.floor(Py/dy)

# Plocz = np.where(ztemp == Pz[:,:,:,:,None])[4]
# Plocz = Plocz.reshape(npcell,nz,ny,nx)
Plocz = np.zeros((npcell,nz,ny,nx))

for ii in range(npcell):
    for i in range(nz):
        for j in range(ny):
            for k in range(nx):
                zz = 0
                for ll in range(nz):
                    zz = zz + dz[ll]
                    if zz >= Pz[ii,i,j,k]:
                        Plocz[ii,i,j,k] = ll-1
                        break            

In [10]:
Plocz[0,0,:,:]

array([[-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1.]])

In [11]:
#cal vel loc

Clocx = (Px - Plocx*dx)/dx
Clocy = (Py - Plocy*dy)/dy
Clocz = np.zeros((npcell,nz,ny,nx))

for ii in range(npcell):
    for i in range(nz):
        for j in range(ny):
            for k in range(nx):
                zz = 0
                temp = int(Plocz[ii,i,j,k]) + 1
                if temp > 0:
                    zz = np.sum(dz[0:temp])
                # print(Pz[ii,i,j,k], zz, temp+1)
                Clocz[ii,i,j,k] = (Pz[ii,i,j,k] - zz)/dz[temp]

In [136]:
Vpx = np.zeros((npcell,nz,ny,nx))
Vpy = np.zeros((npcell,nz,ny,nx))
Vpz = np.zeros((npcell,nz,ny,nx))

for ii in range(npcell):
    for i in range(nz):
        for j in range(ny):
            for k in range(nz):
                
                tempx = Plocx[ii,i,j,k]
                tempy = Plocy[ii,i,j,k]
                tempz = Plocz[ii,i,j,k] + 1
                    
                Vpx[ii,i,j,k] = ((1 - Clocx[ii,i,j,k])*Vx[tempz,tempy,tempx] + \
                                Clocx[ii,i,j,k]*Vx[tempz,tempy,tempx+1])/ \
                                Saturation[tempz,tempy,tempx]/Porosity[tempz,tempy,tempx]
                
                Vpy[ii,i,j,k] = ((1 - Clocy[ii,i,j,k])*Vy[tempz,tempy,tempx] + \
                                Clocy[ii,i,j,k]*Vy[tempz,tempy+1,tempx])/ \
                                Saturation[tempz,tempy,tempx]/Porosity[tempz,tempy,tempx]
                
                Vpz[ii,i,j,k] = ((1 - Clocz[ii,i,j,k])*Vz[tempz,tempy,tempx] + \
                                Clocz[ii,i,j,k]*Vz[tempz+1,tempy,tempx])/ \
                                Saturation[tempz,tempy,tempx]/Porosity[tempz,tempy,tempx]

In [ ]:
Px = Px + Vpx
Py = Py + Vpy
Pz = Pz + Vpz